In [ ]:
import pandas as pd
import numpy as np
import ast
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = movies.merge(credits, on = 'title')
# genres
# id
# keywords
# title
# overview
# cast
# crew
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.isnull().sum()
movies.dropna(inplace = True)
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:

            l.append(i['name'])
            counter +=1
        else:
            break
    return l
movies['cast'] = movies['cast'].apply(convert3)
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['department'] == "Directing":
            L.append(i["name"])
           
        else:
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)


In [ ]:
movies['overview'] = movies['overview'].apply(lambda x :x.split())

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x] )
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x] )
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x] )
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x] )

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply( lambda x: x.lower())
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')


In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')



In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)
# sorted(similarity[0],reverse = 'True')

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances =  similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        

In [164]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
10th & Wolf
The Dark Knight Rises


In [169]:
import pickle

In [171]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))